In [7]:
import torch.nn as nn
import torch


class VGG(nn.Module):
    def __init__(self, num_classes=40, init_weights=False):
        super(VGG, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv1d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv1d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(23552, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

In [8]:
import os
import json
import time
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_transform = transforms.Compose([ transforms.Grayscale(num_output_channels = 1),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5), (0.5))])

    # load npy
    npy_path = "../features/train_npy/Amazon Alexa Echo Dot 1/Amazon Alexa Echo Dot 1_0.npy"
    data = np.load(npy_path)
    data = Image.fromarray(data.astype(np.uint8))
    data = data_transform(data)

    # read class_indict
    json_path = '../idx_label.json'
    with open(json_path, "r") as f:
        class_indict = json.load(f)

    # create model
    model = VGG(num_classes=40).to(device)
    # load model weights
    weights_path = "../VggNet_parameters.pth"
    model.load_state_dict(torch.load(weights_path, map_location=device))

    # prediction
    model.eval()
    with torch.no_grad():
        # predict class
        data = data.reshape(data.shape[0], 1, 1500)
        start_time = time.time()
        output = torch.squeeze(model(data.to(device))).cpu()
        predict = torch.softmax(output, dim=0)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"run time: {execution_time} s")
        predict_cla = torch.argmax(predict).numpy()

    print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                 predict[predict_cla].numpy())
    print(print_res)

if __name__ == '__main__':
    main()

run time: 0.06840729713439941 s
class: Amazon Alexa Echo Dot 1   prob: 1.0
